# 기본환경 설정

In [ ]:
# !pip install faiss-cpu

In [ ]:
from google.colab import userdata
HF_KEY = userdata.get("HF_KEY")

In [ ]:
import huggingface_hub
huggingface_hub.login(HF_KEY)

# 모델 로딩

In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo langchain-community pypdf langchain_huggingface faiss-cpu
!pip install --no-deps unsloth

In [ ]:
from unsloth import FastModel
from langchain.embeddings import HuggingFaceEmbeddings
import torch

In [ ]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 1024*5, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    # device_map = {"": device}
)

In [ ]:
model = FastModel.for_inference(model)
model = model.eval()

# Custom ChatModel 함수

In [ ]:
from typing import Any, Dict, Iterable, Iterator, List, Literal, Optional, Type, Union
from pydantic import BaseModel as PydanticBaseModel
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import AIMessage, AIMessageChunk, HumanMessage, SystemMessage, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatResult, ChatGenerationChunk
from langchain_core.runnables import RunnableLambda
from transformers import TextIteratorStreamer
import threading
import re

In [ ]:
class GemmaChatModel(BaseChatModel):
    def __init__(self, model, tokenizer, max_tokens: int = 512, do_sample: bool = True, temperature: float = 0.7, top_p: float = 0.9, verbose: bool = False, **kwargs: Any):
        super().__init__()
        object.__setattr__(self, "model", model)
        object.__setattr__(self, "tokenizer", tokenizer)
        object.__setattr__(self, "max_tokens", max_tokens)
        object.__setattr__(self, "do_sample", do_sample)
        object.__setattr__(self, "temperature", temperature)
        object.__setattr__(self, "top_p", top_p)
        object.__setattr__(self, "verbose", verbose)
        object.__setattr__(self, "_gen_lock", threading.Lock())

    ### 공통 유틸 ###########################################################################
    @property
    def _llm_type(self) -> str:
        return "gemma-chat"

    def _format_messages(self, messages: List[BaseMessage]) -> str:
        conv: List[Dict[str, str]] = []
        for m in messages:
            if isinstance(m, SystemMessage):
                conv.append({"role": "system", "content": m.content})
            elif isinstance(m, HumanMessage):
                conv.append({"role": "user", "content": m.content})
            elif isinstance(m, AIMessage):
                conv.append({"role": "model", "content": m.content})

        formatted = self.tokenizer.apply_chat_template(
            conv,
            tokenize=False,
            add_generation_prompt=True,  # 어시스턴트 턴 시작만 넣고 종료 토큰은 모델이 생성
        )
        return formatted

    def _apply_stop(self, text: str, stop: Optional[List[str]]) -> str:
        if not stop:
            return text
        cut = len(text)
        for s in stop:
            idx = text.find(s)
            if idx != -1:
                cut = min(cut, idx)
        return text[:cut]

    def _build_gen_kwargs(self, **kwargs: Any) -> Dict[str, Any]:
        pad_id = self.tokenizer.pad_token_id
        if pad_id is None:
            pad_id = self.tokenizer.eos_token_id

        eot_id = None
        try:
            eot_id = self.tokenizer.convert_tokens_to_ids("<end_of_turn>")
            if isinstance(eot_id, list):
                eot_id = None
        except Exception:
            eot_id = None

        return {
            "max_new_tokens": kwargs.get("max_tokens", self.max_tokens),
            "do_sample": kwargs.get("do_sample", self.do_sample),
            "temperature": kwargs.get("temperature", self.temperature),
            "top_p": kwargs.get("top_p", self.top_p),
            "eos_token_id": eot_id or self.tokenizer.eos_token_id,
            "pad_token_id": pad_id,
        }

    ### Invoke ############################################################################
    def _generate(self, messages: List[BaseMessage], stop: Optional[List[str]] = None, run_manager: Optional[Any] = None, **kwargs: Any) -> ChatResult:
        prompt = self._format_messages(messages)
        if getattr(self, "verbose", False):
            print("\n[GemmaChatModel/_generate] ==== FINAL PROMPT ====\n" + prompt + "\n=================================================\n")
        # 기본 stop 시퀀스 (ReAct 루프에서 유용)
        if stop is None:
            stop = ["\nObservation:", "\nFinal Answer:"]
        with self._gen_lock:
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
            gen_kwargs = self._build_gen_kwargs(**kwargs)
            with torch.no_grad():
                outputs = self.model.generate(**inputs, **gen_kwargs)
        in_len = inputs["input_ids"].shape[1]
        gen_tokens = outputs[0][in_len:]
        decoded = self.tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()
        # 안전망: 종료 마커 제거
        decoded = re.sub(r"(?:</s>|<\|endoftext\|>|<end_of_turn>)+\s*$", "", decoded)
        decoded = self._apply_stop(decoded, stop)
        return ChatResult(generations=[ChatGeneration(message=AIMessage(content=decoded))])

    ### Batch #############################################################################
    def _generate_batch(self, messages_list: List[List[BaseMessage]], stop: Optional[List[str]] = None, run_manager: Optional[Any] = None, **kwargs: Any) -> List[ChatResult]:
        """
        여러 개의 대화를 한 번에 패딩 인코딩하여 generate 가속.
        """
        # [CHANGED] 각 대화를 chat template로 포맷
        prompts = [self._format_messages(msgs) for msgs in messages_list]

        # padding=True, truncation=True 로 배치 인코딩
        tokenized = self.tokenizer(
            prompts, return_tensors="pt", padding=True, truncation=True
        )
        tokenized = {k: v.to(self.model.device) for k, v in tokenized.items()}
        attn = tokenized.get("attention_mask", None)

        gen_kwargs = self._build_gen_kwargs(**kwargs)

        # [CHANGED] 기본 stop 시퀀스
        if stop is None:
            stop = ["</s>", "<end_of_turn>", "<|endoftext|>"]

        with torch.no_grad():
            outputs = self.model.generate(**tokenized, **gen_kwargs)

        results: List[ChatResult] = []
        for i in range(len(prompts)):
            # 각 샘플의 프롬프트 길이만큼 잘라서 신규 토큰만 디코딩
            if attn is not None:
                in_len = int(attn[i].sum().item())
            else:
                in_len = tokenized["input_ids"][i].shape[0]

            gen_tokens = outputs[i][in_len:]
            text = self.tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()
            text = re.sub(r"(?:</s>|<\|endoftext\|>|<end_of_turn>)+\s*$", "", text)  # [CHANGED]
            text = self._apply_stop(text, stop)

            results.append(
                ChatResult(generations=[ChatGeneration(message=AIMessage(content=text))])
            )
        return results

    ### Stream ############################################################################
    def _stream(self, messages: List[BaseMessage], stop: Optional[List[str]] = None, run_manager: Optional[Any] = None, **kwargs: Any) -> Iterator[ChatGenerationChunk]:
        """
        LangChain의 Runnable .stream() 에서 호출되는 내부 스트리밍 제너레이터.
        각 토큰 델타를 ChatGenerationChunk(AIMessageChunk) 로 내보냅니다.
        """
        prompt = self._format_messages(messages)
        if getattr(self, "verbose", False):
            print("\n[GemmaChatModel/_stream] ==== FINAL PROMPT ====")
            print(prompt)
            print("================================================\n")

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        gen_kwargs = self._build_gen_kwargs(**kwargs)

        # transformers 스트리머 설정
        streamer = TextIteratorStreamer(
            self.tokenizer, skip_prompt=True, skip_special_tokens=True
        )

        # [CHANGED] 기본 stop 시퀀스
        if stop is None:
            stop = ["</s>", "<end_of_turn>", "<|endoftext|>"]

        # generate를 백그라운드에서 수행
        def _worker():
            with torch.no_grad():
                self.model.generate(**inputs, **gen_kwargs, streamer=streamer)

        th = threading.Thread(target=_worker, daemon=True)
        th.start()

        # 누적 후 stop 시퀀스까지 안전하게 잘라서 내보내기
        buffer = ""
        emitted = 0

        for piece in streamer:
            buffer += piece
            # [CHANGED] 실시간으로 후단 종료 마커 제거(시각적 잔여물 방지)
            tmp = re.sub(r"(?:</s>|<\|endoftext\|>|<end_of_turn>)+\s*$", "", buffer)
            trimmed = self._apply_stop(tmp, stop)

            # 새로 생긴 구간만 델타로 방출
            if len(trimmed) > emitted:
                delta = trimmed[emitted:]
                emitted = len(trimmed)
                yield ChatGenerationChunk(message=AIMessageChunk(content=delta))

            # stop 시퀀스 감지되면 중단
            if stop and len(trimmed) < len(buffer):
                break

        # 스레드 정리(최대한 조용히 종료 대기)
        th.join(timeout=0.1)

    ### Structured output #################################################################
    def _build_json_system_prompt(self, schema_text: str) -> str:
        # 모델이 JSON만 내도록 강하게 지시 (hallucination 방지용 규칙 포함)
        return (
            "You are a strict JSON generator.\n"
            "Return ONLY a single JSON object, no prose, no backticks, no explanations.\n"
            "Do not include trailing commas. Do not include comments.\n"
            "Conform exactly to the following JSON schema (fields, types, required):\n"
            f"{schema_text}\n"
        )

    def _ensure_pydantic(self):
        if PydanticBaseModel is None:
            raise RuntimeError(
                "Pydantic is not available. Install pydantic or pass a dict schema instead of a BaseModel."
            )

    def _schema_to_text(self, schema: Union[Type["PydanticBaseModel"], Dict[str, Any]]) -> str:
        if PydanticBaseModel is not None and isinstance(schema, type) and issubclass(
            schema, PydanticBaseModel
        ):
            try:
                json_schema = schema.model_json_schema()  # pydantic v2
            except Exception:
                json_schema = schema.schema()  # pydantic v1
            return json.dumps(json_schema, ensure_ascii=False, indent=2)
        elif isinstance(schema, dict):
            return json.dumps(schema, ensure_ascii=False, indent=2)
        else:
            raise TypeError(
                "schema must be a Pydantic BaseModel subclass or a dict JSON schema."
            )

    def _parse_structured(self, text: str, schema: Union[Type["PydanticBaseModel"], Dict[str, Any]], include_raw: bool):
        # 코드블록 등 제거 시도(혹시 들어올 경우)
        t = text.strip()
        if t.startswith("```"):
            # ```json ... ``` 또는 ``` ... ```
            t = t.strip("`")
            # 첫 줄에 json 명시가 들어있을 수 있음
            t = "\n".join(
                line for line in t.splitlines() if not line.lower().startswith("json")
            )
        # JSON 파싱
        obj = json.loads(t)

        # Pydantic 검증
        if PydanticBaseModel is not None and isinstance(schema, type) and issubclass(
            schema, PydanticBaseModel
        ):
            validated = (
                schema.model_validate(obj)
                if hasattr(schema, "model_validate")
                else schema.parse_obj(obj)
            )
            return {"parsed": validated, "raw": text} if include_raw else validated
        else:
            # dict 스키마는 별도 검증 없이 반환 (원하면 jsonschema로 검증 가능)
            return {"parsed": obj, "raw": text} if include_raw else obj

    def with_structured_output(self, schema: Union[Type["PydanticBaseModel"], Dict[str, Any]], *, method: Literal["json_mode"] = "json_mode", include_raw: bool = False, system_prefix: Optional[str] = None, deterministic: bool = True):
        schema_text = self._schema_to_text(schema)
        sys_prompt = self._build_json_system_prompt(schema_text)
        if system_prefix:
            sys_prompt = system_prefix.rstrip() + "\n\n" + sys_prompt

        def _invoke(messages_or_any):
            # 입력을 메시지 리스트로 정규화
            if isinstance(messages_or_any, list) and all(
                isinstance(m, BaseMessage) for m in messages_or_any
            ):
                msgs = [SystemMessage(content=sys_prompt)] + messages_or_any
            else:
                # 문자열/딕셔너리 등도 처리
                msgs = [
                    SystemMessage(content=sys_prompt),
                    HumanMessage(content=str(messages_or_any)),
                ]

            # 결정론 옵션
            kw = {}
            if deterministic:
                kw = {"do_sample": False, "temperature": 0.0, "top_p": 1.0}

            # 1차 시도
            result = self._generate(msgs, **kw)
            text = result.generations[0].message.content
            try:
                return self._parse_structured(text, schema, include_raw)
            except Exception:
                # 재시도: 더 강한 지시
                retry_msgs = [
                    SystemMessage(
                        content=sys_prompt + "\nOutput must be valid JSON. Try again."
                    )
                ] + msgs[1:]
                result2 = self._generate(retry_msgs, **kw)
                text2 = result2.generations[0].message.content
                return self._parse_structured(text2, schema, include_raw)

        # Runnable 로 래핑해서 반환 (체인 파이프에 바로 사용 가능)
        return RunnableLambda(_invoke)

In [ ]:
chat_model = GemmaChatModel(model=model, tokenizer=tokenizer, max_tokens=1024*3)

# 기본적인 구성 및 기능 확인 - Runnable: invoke, batch, stream

## invoke

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

In [ ]:
res = chat_model.invoke([
    SystemMessage(content="You are a helpful assistant. Reply in Korean."),
    HumanMessage(content="로컬 Qwen3를 LangChain과 함께 쓰는 법을 요약해줘."),
])

In [ ]:
print(res.content)

## batch

In [ ]:
from langchain_core.messages import HumanMessage

In [ ]:
batch_inputs = [
    [HumanMessage(content="한 문장으로 자기소개해줘.")],
    [HumanMessage(content="파이썬 제너레이터를 간단히 설명해줘.")],
    [HumanMessage(content="서울의 대표 관광지 3곳만 알려줘.")],
]
outs = chat_model.batch(batch_inputs, config={"max_concurrency": 4})

In [ ]:
for o in outs:
    print(">>", o.content)

## stream

In [ ]:
from langchain_core.messages import HumanMessage

In [ ]:
for chunk in chat_model.stream([HumanMessage(content="Qwen3 모델의 장점과 단점을 알려줘.")]):
    print(chunk.content, end="", flush=True)

## Structured Output

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
class MovieInfo(BaseModel):
    title: str = Field(..., description="영화 제목")
    year: int = Field(..., description="개봉 연도")
    genres: list[str] = Field(..., description="장르")
    rating: float = Field(..., description="10점 만점 평점")

structured_llm = chat_model.with_structured_output(MovieInfo)

In [ ]:
result: MovieInfo = structured_llm.invoke(
    "한국 영화 '괴물'의 제목, 개봉연도, 장르들, 대략적 평점을 JSON으로만 답해줘."
)

In [ ]:
print(result)

# Agent

## 공통 유틸/도구

In [ ]:
from langchain.tools import tool

In [ ]:
@tool("multiply")
def multiply_tool(expr: str) -> str:
    """
    두 수의 곱을 반환합니다.
    입력 형식 예: "12 7", "12,7", "12 x 7", "곱하기 12와 7"
    """
    nums = re.findall(r"-?\d+(?:\.\d+)?", expr)
    if len(nums) < 2:
        return "오류: 두 개의 숫자를 찾아야 합니다. 예: '12 7'"
    x, y = float(nums[0]), float(nums[1])
    return str(x * y)

In [ ]:
CITY_TO_COUNTRY: Dict[str, str] = {
    "Seoul": "South Korea",
    "Tokyo": "Japan",
    "Paris": "France",
}

In [ ]:
@tool("lookup_country")
def lookup_country_tool(city_text: str) -> str:
    """
    도시가 속한 국가를 반환합니다. (모르면 'unknown')
    입력은 도시명만 주거나 문장 속에 포함해도 됩니다. 예: "Paris", "도시는 Seoul"
    """
    # 가장 그럴듯한 '단어' 하나를 도시로 취급
    # (간단히 첫 영문 단어를 우선, 없으면 첫 한글/영문 토큰)
    m = re.search(r"[A-Za-z]+", city_text)
    city = m.group(0) if m else city_text.strip().split()[0]
    return CITY_TO_COUNTRY.get(city, "unknown")

In [ ]:
TOOLS = [multiply_tool, lookup_country_tool]

## ReAct : 도구 설명을 보고, 필요한 순간에 툴을 호출하며 추론-행동을 번갈아 수행하는 방식

In [ ]:
from langchain.agents import initialize_agent, AgentType

In [ ]:
react_agent = initialize_agent(
        tools=TOOLS,
        llm=chat_model,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # 간단한 ReAct 프롬프트 내장
        verbose=True,
    )

In [ ]:
output = react_agent.invoke(
    {"input": "12와 7을 곱하고, 'Paris'가 어느 나라 수도인지도 알려줘. 대답은 한국어로 해줘."}
)["output"]

In [ ]:
print(output)

## Conversational : 대화 기록을 활용하여 맥락을 유지하는 방식

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser

In [ ]:
history_store = {}

In [ ]:
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in history_store:
        history_store[session_id] = InMemoryChatMessageHistory()
    return history_store[session_id]

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 유용하고 간결한 한국어 어시스턴트야."),
    MessagesPlaceholder("history"),
    ("human", "{input}"),
])

In [ ]:
base_chain = prompt | chat_model | StrOutputParser()

In [ ]:
conv_agent = RunnableWithMessageHistory(
    base_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
session_cfg = {"configurable": {"session_id": "demo-user-1"}}

In [ ]:
print("[+] first question")
print(conv_agent.invoke({"input": "Gemma 모델에 대해 설명해 주세요."}, config=session_cfg))
print("\n[+] second question")
print(conv_agent.invoke({"input": "어떤 버전이 있다고 했었나요?"}, config=session_cfg))

## Planning : 복잡한 목표를 받아 계획을 세운 뒤 순차적으로 정답을 찾아가는 방식

In [ ]:
# !pip install langchain-experimental

In [ ]:
from langchain_experimental.plan_and_execute import (
    load_chat_planner,
    load_agent_executor,
    PlanAndExecute,
)

In [ ]:
planner = load_chat_planner(chat_model)
executor = load_agent_executor(chat_model, TOOLS, verbose=True)  # 내부적으로 ReAct 계열
plan_agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [ ]:
output = plan_agent.invoke(
    {"input": "12와 7을 곱하고, 'Paris'가 어느 나라 수도인지도 알려줘. 대답은 한국어로 해줘."}
)["output"]

In [ ]:
print(output)

## Planning : 직접 구현

In [ ]:
import json
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import initialize_agent, AgentType

In [ ]:
# 1) Planner: 입력 목표를 보고 JSON 배열로 "간단한 단계 목록"을 생성
PLANNER_PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     "너는 계획 수립 보조자야. 사용자의 목표를 달성하기 위한 "
     "간단하고 실행 가능한 단계(step)만 JSON 배열로 출력해. "
     "설명/코드블록 없이 JSON만. 각 원소는 한국어 한 문장으로."),
    ("human",
     "목표: {goal}\n"
     "출력 형식 예시: [\"수치 A와 B의 곱을 계산한다\", \"도시 X의 국가를 조회한다\", \"결과를 한 문장으로 정리한다\"]")
])

In [ ]:
# 2) Plan-and-Execute 래퍼
class SimplePlanAndExecute:
    def __init__(self, llm, tools):
        self.llm = llm
        self.planner = PLANNER_PROMPT | chat_model | StrOutputParser()
        self.executor = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)  # 스텝 실행은 ReAct로

    def invoke(self, inputs: dict) -> dict:
        goal = inputs["input"] if "input" in inputs else inputs.get("goal", "")
        # (a) 계획 생성
        plan_text = self.planner.invoke({"goal": goal}).strip()
        try:
            steps: List[str] = json.loads(plan_text)
            assert isinstance(steps, list) and all(isinstance(s, str) for s in steps)
        except Exception:
            # 혹시 JSON이 아니면 아주 단순 폴백
            steps = [goal]

        # (b) 스텝별 실행 (각 스텝을 자연어 태스크로 ReAct 에이전트에 던짐)
        observations = []
        for i, step in enumerate(steps, 1):
            out = self.executor.invoke({"input": step})
            obs = out["output"] if isinstance(out, dict) and "output" in out else str(out)
            observations.append(f"[{i}] {step} -> {obs}")

        # (c) 요약/정리 한 번 더 요청(선택)
        summary_prompt = ChatPromptTemplate.from_messages([
            ("system", "다음 관찰 로그를 간결하게 한국어로 종합 보고서 한 문장으로 요약해."),
            ("human", "{log}")
        ])
        summarizer = summary_prompt | self.llm | StrOutputParser()
        summary = summarizer.invoke({"log": "\n".join(observations)})

        return {
            "plan": steps,
            "observations": observations,
            "output": summary.strip(),
        }

In [ ]:
planning_agent = SimplePlanAndExecute(chat_model, TOOLS)

In [ ]:
res = planning_agent.invoke({
    "input": "1) 15와 4를 곱하고 2) 그 결과와 함께 Seoul의 국가를 한 문장으로 정리"
})

In [ ]:
print("\n=== [Planning — no experimental] ===")
print("Plan:", res["plan"])
print("Observations:", *res["observations"], sep="\n")
print("Final:", res["output"])